In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
cd gdrive/My Drive

/content/gdrive/My Drive


In [ ]:
from keras.applications import InceptionV3
from keras.preprocessing.image import ImageDataGenerator 
from keras.models import Sequential
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import os
from skimage import io
from skimage.io import imread,imshow
from skimage import util
from skimage import data,filters
from keras.layers import Conv2D,BatchNormalization,Flatten,MaxPool2D,Dropout,Dense
from keras import layers,Model,models,optimizers,regularizers
from skimage.color import rgb2gray
from skimage.color import gray2rgb
from skimage.measure import compare_ssim
from skimage.transform import resize
from keras.applications import Xception,InceptionV3,ResNet152V2,InceptionResNetV2,ResNet152V2,NASNetLarge,DenseNet201

In [ ]:
def image_data_generator(data_dir,data_augment=False,
                       batch_size=10,
                       target_size=(331,331),
                       class_mode='categorical',
                       subset='training',
                       shuffle=True):
  if data_augment:
      datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
																	 width_shift_range=0.1,
																	 height_shift_range=0.1,
                                   zoom_range=0.11,
																	 shear_range=0.12,
                                   validation_split=0.2,#this is the trick
                                   vertical_flip=True,
                                   horizontal_flip=True)
  else:
      datagen = ImageDataGenerator(rescale=1./255,validation_split=0.2)

  generator = datagen.flow_from_directory(data_dir,
                                          target_size=target_size,
                                          batch_size=batch_size,
                                          shuffle=shuffle,
                                          
                                          class_mode=class_mode,
                                          subset=subset)
  return generator

In [ ]:
train_generator = image_data_generator( "/content/gdrive/My Drive/final/train/r09521227/data",data_augment=True)

Found 4538 images belonging to 4 classes.


In [ ]:
validation_generator = image_data_generator( "/content/gdrive/My Drive/final/train/r09521227/data",data_augment=False,subset='validation')

Found 1133 images belonging to 4 classes.


In [ ]:
#用 NASNetLarge pretrained model 訓練 人工刪除後的資料
conv_base=NASNetLarge(weights='imagenet',input_shape=(331,331,3),include_top=False)
conv_base.trainable=False
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(4, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-5),
              metrics=['acc'])

In [ ]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=40,
      epochs=30,
      validation_data=validation_generator)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/30
40/40 [==============================] - 73s 1s/step - loss: 1.8585 - acc: 0.4414 - val_loss: 0.8440 - val_acc: 0.7043
Epoch 2/30
40/40 [==============================] - 52s 1s/step - loss: 1.5090 - acc: 0.5606 - val_loss: 0.6582 - val_acc: 0.7793
Epoch 3/30
40/40 [==============================] - 53s 1s/step - loss: 0.9812 - acc: 0.7098 - val_loss: 0.5872 - val_acc: 0.7820
Epoch 4/30
40/40 [==============================] - 53s 1s/step - loss: 1.0712 - acc: 0.7093 - val_loss: 0.5234 - val_acc: 0.8111
Epoch 5/30
40/40 [==============================] - 53s 1s/step - loss: 0.8380 - acc: 0.7363 - val_loss: 0.4930 - val_acc: 0.8358
Epoch 6/30
40/40 [==============================] - 52s 1s/step - loss: 1.1078 - acc: 0.6857 - val_loss: 0.4675 - val_acc: 0.8376
Epoch 7/30
40/40 [==============================] - 53s 1s/step - loss: 1.0374 - acc: 0.6976 - val_loss: 0.4455 - val_acc: 0.8367
Epoch 8/30
40/40 [==============================] - 53s 1s/step - loss: 1.1502 - acc: 0.71

In [ ]:
model.save('./model/final_large_add.h5')

In [ ]:
#抓取更多資料(2500)後，用Xception 再次訓練
conv_base = Xception(weights='imagenet', include_top=False, input_shape=(299, 299,3) )
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dropout(0.4))
model.add(layers.Dense(4, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-5),
              metrics=['acc'])

In [ ]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=50,
      epochs=35,
      validation_data=validation_generator)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


50/50 [==============================] - 78s 2s/step - loss: 1.0358 - acc: 0.8731 - val_loss: 1.0592 - val_acc: 0.8348


In [ ]:
submit_dir="/content/gdrive/My Drive/final/test_data"
submit_datagen = ImageDataGenerator(rescale=1./255)
generator = submit_datagen.flow_from_directory(
        submit_dir,
        class_mode=None,
        classes=['no'],
        target_size=(331,331),
        batch_size=1,
        shuffle=False)

Found 2000 images belonging to 1 classes.


In [ ]:
pred=model.predict_generator(generator,verbose=1)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


2000/2000 [==============================] - 63s 30ms/step


In [ ]:
#用 DenseNet201 pre trained model 訓練  只訓練最後頂層 原本資料(完全沒動過)
conv_base=DenseNet201(weights='imagenet',input_shape=(224,224,3),include_top=False)
model = models.Sequential()
conv_base.trainable=False
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(4, activation='sigmoid'))
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-5),
              metrics=['acc'])

In [ ]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=50,
      epochs=35,
      validation_data=validation_generator)

In [ ]:
#開始 瘋狂亂嘗試 ensemble
from keras.models import load_model
model2=load_model('model/final_ception_add.h5')

In [ ]:
model7=load_model('model/final_large_add.h5')

In [ ]:
model3=load_model('model/final_large_2.h5')

In [ ]:
model1=load_model('model/final_IncRes.h5')

In [ ]:
pred_2=model2.predict_generator(generator,verbose=1)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


2000/2000 [==============================] - 605s 299ms/step


In [ ]:
pred_1=model1.predict_generator(generator,verbose=1)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


2000/2000 [==============================] - 62s 30ms/step


In [ ]:
pred_3=model3.predict_generator(generator,verbose=1)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


2000/2000 [==============================] - 105s 50ms/step


In [ ]:
pred_7=model7.predict_generator(generator,verbose=1)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


2000/2000 [==============================] - 106s 51ms/step


In [ ]:
pred_4=pd.read_csv('prediction.csv')
pred_4=np.array(pred_4)

1:InResNet 2:Xception 3:large 4:ResNet201


In [ ]:
predict1=np.argmax(pred_1+pred_2+pred_3+pred_4+pred_7,axis=1)

In [ ]:
predict2=np.argmax(pred_1+pred_2+1.3*pred_3+pred_4+pred_7,axis=1)

In [ ]:
#調整權重 觀察預測不一樣的資料
c=1
for i,j,k in zip(predict1,predict2,generator.filenames):
  if i!=j:
    print(k,j,'to',i)
    print(c)
  c+=1

no/1614caff98e4c9c0920f6e06c7a31dbd.jpg 1 to 2
168
no/18f4af55c78c9777e0ef8383c6b5a64e.jpg 0 to 2
191
no/1e8b5d14ae1947dcd5a3fa1f39887957.jpg 0 to 2
243
no/5301cb076bc51a9274cde05ac736c20e.jpg 1 to 2
646
no/65796192ad98b2c28b21e720b4dcc4ab.jpg 1 to 2
787
no/704bc43c30e8b7d063db147f1fee4f6a.jpg 1 to 2
876
no/7c5d24ce6bb504ad7df6ab751a2015e6.jpg 1 to 2
966
no/80bba2682b770d90036bff3de9c57c93.jpg 3 to 2
989
no/8632fa3a855f7a16df5e0da3f8c4f1ed.jpg 0 to 2
1026
no/98d369fecdfa380e47cd5bc502f72f0f.jpg 1 to 2
1175
no/9fca9f331e18caf0d7511aafa3aaf2bd.jpg 1 to 2
1222
no/a38b89f97394f2dc8d6efb2171a3bc7b.jpg 0 to 2
1255
no/b7f55e0a0433845af4beb7456937b1a7.jpg 1 to 2
1421
no/cec22f532de7fb25f16896ea11f81d16.jpg 1 to 2
1618
no/d20d349139af3a1c186a9edb098a918f.jpg 0 to 2
1647
no/d31353732ef001010dda283c4be39067.jpg 1 to 2
1654
no/e64e5af58a5a48d62914d606611f7be8.jpg 1 to 2
1811
no/ff7fa83be9eecd0128d185ca6c8b2b87.jpg 0 to 2
1995


In [ ]:
#最後輸出
sample=pd.read_csv('sample_output.csv')
names=sample['Name']
submit=pd.DataFrame({'Name':names,'Type':predict1})
submit.set_index('Name',inplace=True)
submit.to_csv('submit.csv',index=True)

In [ ]:
#最後 model ensemble 形式

1.75*Xception(有加)+large(手動刪除資料)+DenseNet201(原本資料集)+InceptionResNet(有加)+Xception(add2500)